In [1]:
import pandas as pd
pd.set_option('display.max_columns', None)

import requests, json
from pprint import pprint

In [2]:
# base url for all FPL API endpoints
base_url = 'https://fantasy.premierleague.com/api/'

# get data from bootstrap-static endpoint
r = requests.get(base_url+'bootstrap-static/').json()

pprint(r, indent=2, depth=1, compact=True)

{ 'element_stats': [...],
  'element_types': [...],
  'elements': [...],
  'events': [...],
  'game_settings': {...},
  'phases': [...],
  'teams': [...],
  'total_players': 7955694}


In [3]:
teams_info = pd.json_normalize(r['teams'])
teams_info = teams_info.loc[:, (teams_info != teams_info.iloc[0]).any()]
teams_info = teams_info.rename(columns = {'name' : 'club_name'})
teams_info = teams_info[['code', 'id', 'club_name', 'strength', 'strength_overall_home','strength_overall_away',
                        'strength_attack_home', 'strength_attack_away', 'strength_defence_home',
                         'strength_defence_away']]

teams_info.head()

code  id    club_name  strength  strength_overall_home  \
0     3   1      Arsenal         4                   1200   
1     7   2  Aston Villa         3                   1100   
2    94   3    Brentford         2                   1020   
3    36   4     Brighton         3                   1100   
4    90   5      Burnley         2                   1060   

   strength_overall_away  strength_attack_home  strength_attack_away  \
0                   1210                  1130                  1150   
1                   1160                  1120                  1140   
2                   1030                  1020                  1030   
3                   1130                  1150                  1190   
4                   1070                  1040                  1090   

   strength_defence_home  strength_defence_away  
0                   1220                   1210  
1                   1080                   1100  
2                   1020                   1030  
3                   1100                   1130  
4                   1040                   1080

In [4]:
pd.json_normalize(r['element_stats'])

label              name
0        Minutes played           minutes
1          Goals scored      goals_scored
2               Assists           assists
3          Clean sheets      clean_sheets
4        Goals conceded    goals_conceded
5             Own goals         own_goals
6       Penalties saved   penalties_saved
7      Penalties missed  penalties_missed
8          Yellow cards      yellow_cards
9             Red cards         red_cards
10                Saves             saves
11                Bonus             bonus
12  Bonus Points System               bps
13            Influence         influence
14           Creativity        creativity
15               Threat            threat
16            ICT Index         ict_index

In [5]:
player_cat = pd.json_normalize(r['element_types'])
player_cat = player_cat[['id', 'singular_name', 'squad_select', 
                         'squad_min_play', 'squad_max_play']]

player_cat

id singular_name  squad_select  squad_min_play  squad_max_play
0   1    Goalkeeper             2               1               1
1   2      Defender             5               3               5
2   3    Midfielder             5               2               5
3   4       Forward             3               1               3

In [6]:
pd.json_normalize(r['elements']).columns

Index(['chance_of_playing_next_round', 'chance_of_playing_this_round', 'code',
       'cost_change_event', 'cost_change_event_fall', 'cost_change_start',
       'cost_change_start_fall', 'dreamteam_count', 'element_type', 'ep_next',
       'ep_this', 'event_points', 'first_name', 'form', 'id', 'in_dreamteam',
       'news', 'news_added', 'now_cost', 'photo', 'points_per_game',
       'second_name', 'selected_by_percent', 'special', 'squad_number',
       'status', 'team', 'team_code', 'total_points', 'transfers_in',
       'transfers_in_event', 'transfers_out', 'transfers_out_event',
       'value_form', 'value_season', 'web_name', 'minutes', 'goals_scored',
       'assists', 'clean_sheets', 'goals_conceded', 'own_goals',
       'penalties_saved', 'penalties_missed', 'yellow_cards', 'red_cards',
       'saves', 'bonus', 'bps', 'influence', 'creativity', 'threat',
       'ict_index', 'influence_rank', 'influence_rank_type', 'creativity_rank',
       'creativity_rank_type', 'threat_rank'

In [7]:
players =  pd.json_normalize(r['elements']).drop(['chance_of_playing_next_round', 'chance_of_playing_this_round','cost_change_event', 'cost_change_event_fall', 'cost_change_start',
                                      'cost_change_start_fall', 'ep_next', 'ep_this', 'event_points', 'id', 'news', 'news_added', 'now_cost', 'photo', 'special', 'squad_number',
                                      'team', 'corners_and_indirect_freekicks_order', 'direct_freekicks_order', 'penalties_order', 'corners_and_indirect_freekicks_text', 'direct_freekicks_text',
                                                  'penalties_text'], axis=1)

players.head()

code  dreamteam_count  element_type      first_name form  in_dreamteam  \
0   80201                0             1           Bernd  1.3         False   
1  115918                0             1      Rúnar Alex  0.0         False   
2   47431                0             3         Willian  0.0         False   
3   54694                0             4  Pierre-Emerick  0.7         False   
4   58822                0             2          Cédric  0.0         False   

  points_per_game      second_name selected_by_percent status  team_code  \
0             1.3             Leno                 1.9      a          3   
1             0.0        Rúnarsson                 1.2      u          3   
2             0.0  Borges Da Silva                 0.1      u          3   
3             1.0       Aubameyang                 1.8      a          3   
4             0.0           Soares                 0.2      a          3   

   total_points  transfers_in  transfers_in_event  transfers_out  \
0             4         35419               27628          72110   
1             0         19017                8732          29117   
2             0           914                  50          17118   
3             2         77522               64699         160683   
4             0          6488                4408          10890   

   transfers_out_event value_form value_season    web_name  minutes  \
0                23944        0.3          0.8        Leno      270   
1                16943        0.0          0.0   Rúnarsson        0   
2                 3168        0.0          0.0     Willian        0   
3                30748        0.1          0.2  Aubameyang       87   
4                 4794        0.0          0.0      Cédric      180   

   goals_scored  assists  clean_sheets  goals_conceded  own_goals  \
0             0        0             0               9          0   
1             0        0             0               0          0   
2             0        0             0               0          0   
3             0        0             0               4          0   
4             0        0             0               7          0   

   penalties_saved  penalties_missed  yellow_cards  red_cards  saves  bonus  \
0                0                 0             0          0      9      0   
1                0                 0             0          0      0      0   
2                0                 0             0          0      0      0   
3                0                 0             0          0      0      0   
4                0                 0             1          0      0      0   

   bps influence creativity threat ict_index  influence_rank  \
0   48      79.0        0.0    0.0       7.9              41   
1    0       0.0        0.0    0.0       0.0             354   
2    0       0.0        0.0    0.0       0.0             584   
3    5       5.0        2.7   19.0       2.6             286   
4   16      21.4       21.8    0.0       4.3             202   

   influence_rank_type  creativity_rank  creativity_rank_type  threat_rank  \
0                    6              489                    51          462   
1                   34              343                    17          288   
2                  238              584                   239          581   
3                   31              253                    35          144   
4                   84              143                    35          324   

   threat_rank_type  ict_index_rank  ict_index_rank_type  
0                49             161                    7  
1                14             364                   34  
2               238             585                  239  
3                31             271                   34  
4               121             227                   80

In [8]:
players.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 594 entries, 0 to 593
Data columns (total 44 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   code                  594 non-null    int64 
 1   dreamteam_count       594 non-null    int64 
 2   element_type          594 non-null    int64 
 3   first_name            594 non-null    object
 4   form                  594 non-null    object
 5   in_dreamteam          594 non-null    bool  
 6   points_per_game       594 non-null    object
 7   second_name           594 non-null    object
 8   selected_by_percent   594 non-null    object
 9   status                594 non-null    object
 10  team_code             594 non-null    int64 
 11  total_points          594 non-null    int64 
 12  transfers_in          594 non-null    int64 
 13  transfers_in_event    594 non-null    int64 
 14  transfers_out         594 non-null    int64 
 15  transfers_out_event   594 non-null    in

In [9]:
# Aggregating name

players['full_name'] = players[['first_name', 'second_name']].agg(' '.join, axis=1)
players = players.drop(['first_name', 'second_name'], axis=1)
players.head()

code  dreamteam_count  element_type form  in_dreamteam points_per_game  \
0   80201                0             1  1.3         False             1.3   
1  115918                0             1  0.0         False             0.0   
2   47431                0             3  0.0         False             0.0   
3   54694                0             4  0.7         False             1.0   
4   58822                0             2  0.0         False             0.0   

  selected_by_percent status  team_code  total_points  transfers_in  \
0                 1.9      a          3             4         35419   
1                 1.2      u          3             0         19017   
2                 0.1      u          3             0           914   
3                 1.8      a          3             2         77522   
4                 0.2      a          3             0          6488   

   transfers_in_event  transfers_out  transfers_out_event value_form  \
0               27628          72110                23944        0.3   
1                8732          29117                16943        0.0   
2                  50          17118                 3168        0.0   
3               64699         160683                30748        0.1   
4                4408          10890                 4794        0.0   

  value_season    web_name  minutes  goals_scored  assists  clean_sheets  \
0          0.8        Leno      270             0        0             0   
1          0.0   Rúnarsson        0             0        0             0   
2          0.0     Willian        0             0        0             0   
3          0.2  Aubameyang       87             0        0             0   
4          0.0      Cédric      180             0        0             0   

   goals_conceded  own_goals  penalties_saved  penalties_missed  yellow_cards  \
0               9          0                0                 0             0   
1               0          0                0                 0             0   
2               0          0                0                 0             0   
3               4          0                0                 0             0   
4               7          0                0                 0             1   

   red_cards  saves  bonus  bps influence creativity threat ict_index  \
0          0      9      0   48      79.0        0.0    0.0       7.9   
1          0      0      0    0       0.0        0.0    0.0       0.0   
2          0      0      0    0       0.0        0.0    0.0       0.0   
3          0      0      0    5       5.0        2.7   19.0       2.6   
4          0      0      0   16      21.4       21.8    0.0       4.3   

   influence_rank  influence_rank_type  creativity_rank  creativity_rank_type  \
0              41                    6              489                    51   
1             354                   34              343                    17   
2             584                  238              584                   239   
3             286                   31              253                    35   
4             202                   84              143                    35   

   threat_rank  threat_rank_type  ict_index_rank  ict_index_rank_type  \
0          462                49             161                    7   
1          288                14             364                   34   
2          581               238             585                  239   
3          144                31             271                   34   
4          324               121             227                   80   

                   full_name  
0                 Bernd Leno  
1       Rúnar Alex Rúnarsson  
2    Willian Borges Da Silva  
3  Pierre-Emerick Aubameyang  
4              Cédric Soares

In [10]:
# Attaching teams_info to players

players = pd.merge(left = players,
         right = teams_info,
         left_on = 'team_code',
         right_on = 'code',
         suffixes=('_player', '_team')).drop(['team_code', 'code_team'] ,axis=1)

players.head()

code_player  dreamteam_count  element_type form  in_dreamteam  \
0        80201                0             1  1.3         False   
1       115918                0             1  0.0         False   
2        47431                0             3  0.0         False   
3        54694                0             4  0.7         False   
4        58822                0             2  0.0         False   

  points_per_game selected_by_percent status  total_points  transfers_in  \
0             1.3                 1.9      a             4         35419   
1             0.0                 1.2      u             0         19017   
2             0.0                 0.1      u             0           914   
3             1.0                 1.8      a             2         77522   
4             0.0                 0.2      a             0          6488   

   transfers_in_event  transfers_out  transfers_out_event value_form  \
0               27628          72110                23944        0.3   
1                8732          29117                16943        0.0   
2                  50          17118                 3168        0.0   
3               64699         160683                30748        0.1   
4                4408          10890                 4794        0.0   

  value_season    web_name  minutes  goals_scored  assists  clean_sheets  \
0          0.8        Leno      270             0        0             0   
1          0.0   Rúnarsson        0             0        0             0   
2          0.0     Willian        0             0        0             0   
3          0.2  Aubameyang       87             0        0             0   
4          0.0      Cédric      180             0        0             0   

   goals_conceded  own_goals  penalties_saved  penalties_missed  yellow_cards  \
0               9          0                0                 0             0   
1               0          0                0                 0             0   
2               0          0                0                 0             0   
3               4          0                0                 0             0   
4               7          0                0                 0             1   

   red_cards  saves  bonus  bps influence creativity threat ict_index  \
0          0      9      0   48      79.0        0.0    0.0       7.9   
1          0      0      0    0       0.0        0.0    0.0       0.0   
2          0      0      0    0       0.0        0.0    0.0       0.0   
3          0      0      0    5       5.0        2.7   19.0       2.6   
4          0      0      0   16      21.4       21.8    0.0       4.3   

   influence_rank  influence_rank_type  creativity_rank  creativity_rank_type  \
0              41                    6              489                    51   
1             354                   34              343                    17   
2             584                  238              584                   239   
3             286                   31              253                    35   
4             202                   84              143                    35   

   threat_rank  threat_rank_type  ict_index_rank  ict_index_rank_type  \
0          462                49             161                    7   
1          288                14             364                   34   
2          581               238             585                  239   
3          144                31             271                   34   
4          324               121             227                   80   

                   full_name  id club_name  strength  strength_overall_home  \
0                 Bernd Leno   1   Arsenal         4                   1200   
1       Rúnar Alex Rúnarsson   1   Arsenal         4                   1200   
2    Willian Borges Da Silva   1   Arsenal         4                   1200   
3  Pierre-Emerick Aubameyang   1   Arsenal         4                   1200   


In [11]:
# Attaching player category to players

players = pd.merge(left = players,
         right = player_cat,
         left_on = 'element_type',
         right_on = 'id',
         suffixes=('_player', '_player_type'),).drop(['element_type', 'id_player_type'] ,axis=1)

players.head()

code_player  dreamteam_count form  in_dreamteam points_per_game  \
0        80201                0  1.3         False             1.3   
1       115918                0  0.0         False             0.0   
2       463748                0  0.0         False             0.0   
3       225321                0  0.0         False             0.0   
4       220682                0  0.0         False             0.0   

  selected_by_percent status  total_points  transfers_in  transfers_in_event  \
0                 1.9      a             4         35419               27628   
1                 1.2      u             0         19017                8732   
2                 0.5      a             0         25123               17448   
3                 0.2      a             0         12838                8005   
4                 0.1      a             0          5842                5842   

   transfers_out  transfers_out_event value_form value_season   web_name  \
0          72110                23944        0.3          0.8       Leno   
1          29117                16943        0.0          0.0  Rúnarsson   
2           5538                 4025        0.0          0.0       Hein   
3           4428                 3333        0.0          0.0   Ramsdale   
4           1318                 1318        0.0          0.0    Okonkwo   

   minutes  goals_scored  assists  clean_sheets  goals_conceded  own_goals  \
0      270             0        0             0               9          0   
1        0             0        0             0               0          0   
2        0             0        0             0               0          0   
3        0             0        0             0               0          0   
4        0             0        0             0               0          0   

   penalties_saved  penalties_missed  yellow_cards  red_cards  saves  bonus  \
0                0                 0             0          0      9      0   
1                0                 0             0          0      0      0   
2                0                 0             0          0      0      0   
3                0                 0             0          0      0      0   
4                0                 0             0          0      0      0   

   bps influence creativity threat ict_index  influence_rank  \
0   48      79.0        0.0    0.0       7.9              41   
1    0       0.0        0.0    0.0       0.0             354   
2    0       0.0        0.0    0.0       0.0             375   
3    0       0.0        0.0    0.0       0.0             461   
4    0       0.0        0.0    0.0       0.0             362   

   influence_rank_type  creativity_rank  creativity_rank_type  threat_rank  \
0                    6              489                    51          462   
1                   34              343                    17          288   
2                   41              364                    24          312   
3                   59              455                    47          420   
4                   36              351                    19          296   

   threat_rank_type  ict_index_rank  ict_index_rank_type  \
0                49             161                    7   
1                14             364                   34   
2                21             385                   41   
3                45             467                   59   
4                16             372                   36   

              full_name  id_player club_name  strength  strength_overall_home  \
0            Bernd Leno          1   Arsenal         4                   1200   
1  Rúnar Alex Rúnarsson          1   Arsenal         4                   1200   
2             Karl Hein          1   Arsenal         4                   1200   
3        Aaron Ramsdale          1   Arsenal         4                   1200   
4        Arthur Okonkwo          1   Arsenal         4                   1200   

In [12]:
# next 3 fixtures

r = requests.get(base_url+'fixtures/').json()

matches = pd.json_normalize(r)
matches.kickoff_time = pd.to_datetime(matches.kickoff_time)

matches = matches[matches.kickoff_time>=pd.Timestamp.utcnow()][['event', 
                                                      'team_h', 'team_a', 'team_h_difficulty', 'team_a_difficulty']].head(30)

dict_team_name = dict()
for i in range(teams_info.shape[0]):
    dict_team_name[teams_info.loc[i, 'id']] = teams_info.loc[i, 'club_name']

matches.team_h = matches.team_h.map(dict_team_name) 
matches.team_a = matches.team_a.map(dict_team_name) 

team_home = matches.drop('team_a', axis=1)
team_home = team_home.rename(columns = {'team_h' : 'team'})
team_home['h_a'] = 'h'

team_away = matches.drop('team_h', axis=1)
team_away = team_away.rename(columns = {'team_a' : 'team'})
team_away['h_a'] = 'a'

fixture_next_3 = pd.concat([team_home, team_away],  axis=0).reset_index(drop=True)
fixture_next_3.head()

event            team  team_h_difficulty  team_a_difficulty h_a
0      4  Crystal Palace                  3                  2   h
1      4         Arsenal                  2                  3   h
2      4       Brentford                  2                  2   h
3      4       Leicester                  4                  3   h
4      4         Man Utd                  2                  4   h

In [13]:
# Next first

matches1 = matches[['event', 'team_h', 'team_a']].reset_index(drop=True)

next_first1 = pd.merge(left = fixture_next_3[fixture_next_3.event == fixture_next_3.event.min()],
         right = matches1[matches1.event == matches1.event.min()],
         left_on = 'team',
         right_on = 'team_h', how='inner').drop(['event_y', 'team_h'], axis=1)
next_first1 = next_first1.rename(columns = {'team_a':'next_1', 'event_x':'event'})



next_first2 = pd.merge(left = fixture_next_3[fixture_next_3.event == fixture_next_3.event.min()],
         right = matches1[matches1.event == matches1.event.min()],
         left_on = 'team',
         right_on = 'team_a', how='inner').drop(['event_y', 'team_a'], axis=1)
next_first2 = next_first2.rename(columns = {'team_h':'next_1', 'event_x':'event'})

next_first_final = pd.concat([next_first1, next_first2], axis=0).reset_index(drop=True)




# Next second

next_second1 = pd.merge(left = fixture_next_3[fixture_next_3.event == fixture_next_3.event.median()],
         right = matches1[matches1.event == matches1.event.median()],
         left_on = 'team',
         right_on = 'team_h', how='inner').drop(['event_y', 'team_h'], axis=1)
next_second1 = next_second1.rename(columns = {'team_a':'next_2', 'event_x':'event'})



next_second2 = pd.merge(left = fixture_next_3[fixture_next_3.event == fixture_next_3.event.median()],
         right = matches1[matches1.event == matches1.event.median()],
         left_on = 'team',
         right_on = 'team_a', how='inner').drop(['event_y', 'team_a'], axis=1)
next_second2 = next_second2.rename(columns = {'team_h':'next_2', 'event_x':'event'})

next_second_final = pd.concat([next_second1, next_second2], axis=0).reset_index(drop=True)




# Next Third

next_third1 = pd.merge(left = fixture_next_3[fixture_next_3.event == fixture_next_3.event.max()],
         right = matches1[matches1.event == matches1.event.max()],
         left_on = 'team',
         right_on = 'team_h', how='inner').drop(['event_y', 'team_h'], axis=1)
next_third1 = next_third1.rename(columns = {'team_a':'next_3', 'event_x':'event'})



next_third2 = pd.merge(left = fixture_next_3[fixture_next_3.event == fixture_next_3.event.max()],
         right = matches1[matches1.event == matches1.event.max()],
         left_on = 'team',
         right_on = 'team_a', how='inner').drop(['event_y', 'team_a'], axis=1)
next_third2 = next_third2.rename(columns = {'team_h':'next_3', 'event_x':'event'})

next_third_final = pd.concat([next_third1, next_third2], axis=0).reset_index(drop=True)



# merging Next 1 and 2

first_second = pd.merge(left = next_first_final,
         right = next_second_final,
         left_on = 'team',
         right_on = 'team',
         suffixes = ('_next_first', '_next_second'))



# merging all

next_three_all = pd.merge(left = first_second,
         right = next_third_final,
         left_on = 'team',
         right_on = 'team',
         suffixes = ('_next_first', '_next_third'))

next_three_all = next_three_all.rename(columns = {'event' : 'event_third',
                                                 'team_h_difficulty' : 'team_h_difficulty_third',
                                                 'team_a_difficulty' : 'team_a_difficulty_third',
                                                 'h_a' : 'h_a_third'})
next_three_all[['team', 'next_1', 'next_2', 'next_3']].head()

team     next_1     next_2       next_3
0  Crystal Palace      Spurs  Liverpool     Brighton
1         Arsenal    Norwich    Burnley        Spurs
2       Brentford   Brighton     Wolves    Liverpool
3       Leicester   Man City   Brighton      Burnley
4         Man Utd  Newcastle   West Ham  Aston Villa

In [14]:
players = pd.merge(left = players,
         right = next_three_all,
         left_on = 'club_name',
         right_on = 'team')

players.head()

code_player  dreamteam_count form  in_dreamteam points_per_game  \
0        80201                0  1.3         False             1.3   
1       115918                0  0.0         False             0.0   
2       463748                0  0.0         False             0.0   
3       225321                0  0.0         False             0.0   
4       220682                0  0.0         False             0.0   

  selected_by_percent status  total_points  transfers_in  transfers_in_event  \
0                 1.9      a             4         35419               27628   
1                 1.2      u             0         19017                8732   
2                 0.5      a             0         25123               17448   
3                 0.2      a             0         12838                8005   
4                 0.1      a             0          5842                5842   

   transfers_out  transfers_out_event value_form value_season   web_name  \
0          72110                23944        0.3          0.8       Leno   
1          29117                16943        0.0          0.0  Rúnarsson   
2           5538                 4025        0.0          0.0       Hein   
3           4428                 3333        0.0          0.0   Ramsdale   
4           1318                 1318        0.0          0.0    Okonkwo   

   minutes  goals_scored  assists  clean_sheets  goals_conceded  own_goals  \
0      270             0        0             0               9          0   
1        0             0        0             0               0          0   
2        0             0        0             0               0          0   
3        0             0        0             0               0          0   
4        0             0        0             0               0          0   

   penalties_saved  penalties_missed  yellow_cards  red_cards  saves  bonus  \
0                0                 0             0          0      9      0   
1                0                 0             0          0      0      0   
2                0                 0             0          0      0      0   
3                0                 0             0          0      0      0   
4                0                 0             0          0      0      0   

   bps influence creativity threat ict_index  influence_rank  \
0   48      79.0        0.0    0.0       7.9              41   
1    0       0.0        0.0    0.0       0.0             354   
2    0       0.0        0.0    0.0       0.0             375   
3    0       0.0        0.0    0.0       0.0             461   
4    0       0.0        0.0    0.0       0.0             362   

   influence_rank_type  creativity_rank  creativity_rank_type  threat_rank  \
0                    6              489                    51          462   
1                   34              343                    17          288   
2                   41              364                    24          312   
3                   59              455                    47          420   
4                   36              351                    19          296   

   threat_rank_type  ict_index_rank  ict_index_rank_type  \
0                49             161                    7   
1                14             364                   34   
2                21             385                   41   
3                45             467                   59   
4                16             372                   36   

              full_name  id_player club_name  strength  strength_overall_home  \
0            Bernd Leno          1   Arsenal         4                   1200   
1  Rúnar Alex Rúnarsson          1   Arsenal         4                   1200   
2             Karl Hein          1   Arsenal         4                   1200   
3        Aaron Ramsdale          1   Arsenal         4                   1200   
4        Arthur Okonkwo          1   Arsenal         4                   1200   

# Checks

### Transfer In-Out

In [15]:
# Top In

players.sort_values(by = 'transfers_in_event',
                    axis = 0,
                    ascending  = False).head(10)[['full_name','influence', 'creativity',
                                                  'threat', 'ict_index', 'minutes',
                                                  'points_per_game', 'club_name',
                                                  'next_1', 'next_2', 'next_3']]

full_name influence creativity threat  \
378  Cristiano Ronaldo dos Santos Aveiro       0.0        0.0    0.0   
346                        Ferran Torres      88.4       23.5  137.0   
551                      Michail Antonio     207.0       93.1  247.0   
231                         Demarai Gray      94.4       56.0   68.0   
543                        Saïd Benrahma     121.6       63.4  107.0   
371                      Mason Greenwood     112.6       64.8   89.0   
119                          Shane Duffy     119.2        0.9  114.0   
493                      Sergio Reguilón      43.4       47.0   12.0   
318                           Diogo Jota      87.2       33.0  136.0   
178                        Marcos Alonso      99.6      105.1   39.0   

    ict_index  minutes points_per_game  club_name          next_1  \
378       0.0        0             0.0    Man Utd       Newcastle   
346      24.9      240             7.7   Man City       Leicester   
551      54.7      268            13.3   West Ham     Southampton   
231      21.9      244             6.3    Everton         Burnley   
543      29.2      264             8.7   West Ham     Southampton   
371      26.6      269             8.7    Man Utd       Newcastle   
119      23.5      270             5.7   Brighton       Brentford   
493      10.3      270             7.3      Spurs  Crystal Palace   
318      25.7      187             5.7  Liverpool           Leeds   
178      24.5      265             7.7    Chelsea     Aston Villa   

             next_2          next_3  
378        West Ham     Aston Villa  
346     Southampton         Chelsea  
551         Man Utd           Leeds  
231     Aston Villa         Norwich  
543         Man Utd           Leeds  
371        West Ham     Aston Villa  
119       Leicester  Crystal Palace  
493         Chelsea         Arsenal  
318  Crystal Palace       Brentford  
178           Spurs        Man City

In [16]:
# Top Out

players.sort_values(by = 'transfers_out_event',
                    axis = 0,
                    ascending  = False).head(10)[['full_name','influence', 'creativity',
                                                  'threat', 'ict_index', 'minutes',
                                                  'points_per_game', 'club_name',
                                                  'next_1', 'next_2', 'next_3']]

full_name influence creativity threat ict_index  \
367  Bruno Miguel Borges Fernandes     125.0       91.9  111.0      32.8   
56                      Danny Ings      89.0       15.6   84.0      18.9   
37               Emiliano Martínez      50.2        0.0    0.0       5.0   
331          Konstantinos Tsimikas      51.2       75.9   39.0      16.6   
474                  Heung-Min Son      77.6       57.1  112.0      24.7   
380                      Luke Shaw      26.0       81.2    6.0      11.3   
175                  Romelu Lukaku      51.8       60.9  165.0      27.8   
84                      Ivan Toney      41.6       32.3  103.0      17.7   
234          Dominic Calvert-Lewin     118.4       39.8  178.0      33.7   
498                     Ben Foster       0.0        0.0    0.0       0.0   

     minutes points_per_game    club_name          next_1          next_2  \
367      270             7.7      Man Utd       Newcastle        West Ham   
56       270             6.3  Aston Villa         Chelsea         Everton   
37       270             3.0  Aston Villa         Chelsea         Everton   
331      184             6.0    Liverpool           Leeds  Crystal Palace   
474      248             7.7        Spurs  Crystal Palace         Chelsea   
380      270             3.7      Man Utd       Newcastle        West Ham   
175      180             5.0      Chelsea     Aston Villa           Spurs   
84       270             3.3    Brentford        Brighton          Wolves   
234      248             7.7      Everton         Burnley     Aston Villa   
498        0             0.0      Watford          Wolves         Norwich   

          next_3  
367  Aston Villa  
56       Man Utd  
37       Man Utd  
331    Brentford  
474      Arsenal  
380  Aston Villa  
175     Man City  
84     Liverpool  
234      Norwich  
498    Newcastle

In [17]:
# GK In

players[players.singular_name == 'Goalkeeper'].sort_values(by = 'transfers_in_event',
                    axis = 0,
                    ascending  = False).head(5)[['full_name','influence', 'creativity',
                                                  'threat', 'ict_index', 'minutes',
                                                  'points_per_game', 'club_name',
                                                 'next_1', 'next_2', 'next_3']]

full_name influence creativity threat ict_index  minutes  \
471        Hugo Lloris      93.8       10.0    0.0      10.4      270   
67   David Raya Martin      86.6        0.0    0.0       8.6      270   
99      Robert Sánchez      54.8       10.0    0.0       6.4      270   
360       David de Gea      67.6        0.0    0.0       6.7      270   
248  Kasper Schmeichel      65.6        0.0    0.0       6.6      270   

    points_per_game  club_name          next_1     next_2          next_3  
471             9.0      Spurs  Crystal Palace    Chelsea         Arsenal  
67              5.7  Brentford        Brighton     Wolves       Liverpool  
99              3.3   Brighton       Brentford  Leicester  Crystal Palace  
360             4.0    Man Utd       Newcastle   West Ham     Aston Villa  
248             4.3  Leicester        Man City   Brighton         Burnley

In [18]:
# GK Out

players[players.singular_name == 'Goalkeeper'].sort_values(by = 'transfers_out_event',
                    axis = 0,
                    ascending  = False).head(5)[['full_name','influence', 'creativity',
                                                  'threat', 'ict_index', 'minutes',
                                                  'points_per_game', 'club_name',
                                                 'next_1', 'next_2', 'next_3']]

full_name influence creativity threat ict_index  minutes  \
37           Emiliano Martínez      50.2        0.0    0.0       5.0      270   
498                 Ben Foster       0.0        0.0    0.0       0.0        0   
99              Robert Sánchez      54.8       10.0    0.0       6.4      270   
248          Kasper Schmeichel      65.6        0.0    0.0       6.6      270   
335  Ederson Santana de Moraes      16.2        0.0    0.0       1.6      270   

    points_per_game    club_name     next_1       next_2          next_3  
37              3.0  Aston Villa    Chelsea      Everton         Man Utd  
498             0.0      Watford     Wolves      Norwich       Newcastle  
99              3.3     Brighton  Brentford    Leicester  Crystal Palace  
248             4.3    Leicester   Man City     Brighton         Burnley  
335             4.7     Man City  Leicester  Southampton         Chelsea

In [19]:
# Defender In

players[players.singular_name == 'Defender'].sort_values(by = 'transfers_in_event',
                    axis = 0,
                    ascending  = False).head(5)[['full_name','influence', 'creativity',
                                                  'threat', 'ict_index', 'minutes',
                                                  'points_per_game', 'club_name',
                                                 'next_1', 'next_2', 'next_3']]

full_name influence creativity threat ict_index  minutes  \
119       Shane Duffy     119.2        0.9  114.0      23.5      270   
493   Sergio Reguilón      43.4       47.0   12.0      10.3      270   
178     Marcos Alonso      99.6      105.1   39.0      24.5      265   
87     Pontus Jansson      53.4       20.6    4.0       7.9      270   
433  Brandon Williams      14.0        3.1    0.0       1.7       90   

    points_per_game  club_name          next_1     next_2          next_3  
119             5.7   Brighton       Brentford  Leicester  Crystal Palace  
493             7.3      Spurs  Crystal Palace    Chelsea         Arsenal  
178             7.7    Chelsea     Aston Villa      Spurs        Man City  
87              7.3  Brentford        Brighton     Wolves       Liverpool  
433             1.0    Norwich         Arsenal    Watford         Everton

In [20]:
# Defender Out

players[players.singular_name == 'Defender'].sort_values(by = 'transfers_out_event',
                    axis = 0,
                    ascending  = False).head(5)[['full_name','influence', 'creativity',
                                                  'threat', 'ict_index', 'minutes',
                                                  'points_per_game', 'club_name',
                                                 'next_1', 'next_2', 'next_3']]

full_name influence creativity threat ict_index  minutes  \
331   Konstantinos Tsimikas      51.2       75.9   39.0      16.6      184   
380               Luke Shaw      26.0       81.2    6.0      11.3      270   
183             Reece James      85.4       71.6   67.0      22.4      160   
33                Ben White      10.8        0.5    0.0       1.1       90   
327  Trent Alexander-Arnold      99.4      226.1   60.0      38.6      270   

    points_per_game  club_name       next_1          next_2       next_3  
331             6.0  Liverpool        Leeds  Crystal Palace    Brentford  
380             3.7    Man Utd    Newcastle        West Ham  Aston Villa  
183             6.7    Chelsea  Aston Villa           Spurs     Man City  
33              1.0    Arsenal      Norwich         Burnley        Spurs  
327             7.3  Liverpool        Leeds  Crystal Palace    Brentford

In [21]:
# Midfielder In

players[players.singular_name == 'Midfielder'].sort_values(by = 'transfers_in_event',
                    axis = 0,
                    ascending  = False).head(5)[['full_name','influence', 'creativity',
                                                  'threat', 'ict_index', 'minutes',
                                                  'points_per_game', 'club_name',
                                                 'next_1', 'next_2', 'next_3']]

full_name influence creativity threat ict_index  minutes  \
346    Ferran Torres      88.4       23.5  137.0      24.9      240   
231     Demarai Gray      94.4       56.0   68.0      21.9      244   
543    Saïd Benrahma     121.6       63.4  107.0      29.2      264   
371  Mason Greenwood     112.6       64.8   89.0      26.6      269   
318       Diogo Jota      87.2       33.0  136.0      25.7      187   

    points_per_game  club_name       next_1          next_2       next_3  
346             7.7   Man City    Leicester     Southampton      Chelsea  
231             6.3    Everton      Burnley     Aston Villa      Norwich  
543             8.7   West Ham  Southampton         Man Utd        Leeds  
371             8.7    Man Utd    Newcastle        West Ham  Aston Villa  
318             5.7  Liverpool        Leeds  Crystal Palace    Brentford

In [22]:
# Midfielder Out

players[players.singular_name == 'Midfielder'].sort_values(by = 'transfers_out_event',
                    axis = 0,
                    ascending  = False).head(5)[['full_name','influence', 'creativity',
                                                  'threat', 'ict_index', 'minutes',
                                                  'points_per_game', 'club_name',
                                                 'next_1', 'next_2', 'next_3']]

full_name influence creativity threat ict_index  \
367  Bruno Miguel Borges Fernandes     125.0       91.9  111.0      32.8   
474                  Heung-Min Son      77.6       57.1  112.0      24.7   
286           Raphael Dias Belloli      64.4       73.9   88.0      22.7   
338                  Jack Grealish      89.2      123.5  146.0      35.9   
364                     Paul Pogba     108.6       98.8  102.0      31.0   

     minutes points_per_game club_name          next_1       next_2  \
367      270             7.7   Man Utd       Newcastle     West Ham   
474      248             7.7     Spurs  Crystal Palace      Chelsea   
286      270             3.3     Leeds       Liverpool    Newcastle   
338      254             5.0  Man City       Leicester  Southampton   
364      254             7.3   Man Utd       Newcastle     West Ham   

          next_3  
367  Aston Villa  
474      Arsenal  
286     West Ham  
338      Chelsea  
364  Aston Villa

In [23]:
# Forward In

players[players.singular_name == 'Forward'].sort_values(by = 'transfers_in_event',
                    axis = 0,
                    ascending  = False).head(5)[['full_name','influence', 'creativity',
                                                  'threat', 'ict_index', 'minutes',
                                                  'points_per_game', 'club_name',
                                                 'next_1', 'next_2', 'next_3']]

full_name influence creativity threat  \
378  Cristiano Ronaldo dos Santos Aveiro       0.0        0.0    0.0   
551                      Michail Antonio     207.0       93.1  247.0   
175                        Romelu Lukaku      51.8       60.9  165.0   
522                      Emmanuel Dennis      74.2       51.5  132.0   
351            Gabriel Fernando de Jesus     102.6      116.8   59.0   

    ict_index  minutes points_per_game club_name       next_1       next_2  \
378       0.0        0             0.0   Man Utd    Newcastle     West Ham   
551      54.7      268            13.3  West Ham  Southampton      Man Utd   
175      27.8      180             5.0   Chelsea  Aston Villa        Spurs   
522      25.8      245             5.0   Watford       Wolves      Norwich   
351      27.9      170             8.0  Man City    Leicester  Southampton   

          next_3  
378  Aston Villa  
551        Leeds  
175     Man City  
522    Newcastle  
351      Chelsea

In [27]:
# Forward Out

players[players.singular_name == 'Forward'].sort_values(by = 'transfers_out_event',
                    axis = 0,
                    ascending  = False).head(5)[['full_name','influence', 'creativity',
                                                  'threat', 'ict_index', 'minutes',
                                                  'points_per_game', 'club_name',
                                                 'next_1', 'next_2', 'next_3']]

full_name influence creativity threat ict_index  minutes  \
56               Danny Ings      89.0       15.6   84.0      18.9      270   
175           Romelu Lukaku      51.8       60.9  165.0      27.8      180   
84               Ivan Toney      41.6       32.3  103.0      17.7      270   
234   Dominic Calvert-Lewin     118.4       39.8  178.0      33.7      248   
235  Richarlison de Andrade      80.2       72.6   93.0      24.5      268   

    points_per_game    club_name       next_1       next_2     next_3  
56              6.3  Aston Villa      Chelsea      Everton    Man Utd  
175             5.0      Chelsea  Aston Villa        Spurs   Man City  
84              3.3    Brentford     Brighton       Wolves  Liverpool  
234             7.7      Everton      Burnley  Aston Villa    Norwich  
235             4.7      Everton      Burnley  Aston Villa    Norwich